# Auswertung der Mensa-Umfrage
zunächst müssen die relevanten libraries importiert werden.

In [ ]:
import pandas as pd
import numpy
import os
# import plotly.express as px
# import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns




Zunächst sollten die Daten der Umfrage aus der csv-Datei gelesen werden.

In [ ]:
cwdir = os.getcwd().split("\\bin")[0]
datadirpath = cwdir + "\\20231208-2140"
filepath = datadirpath + "\\Mensa-Umfrage - survey-results 20231208-2140.csv"
answerpath = cwdir + "\\Mensa-Umfrage - Antwort-schlüssel-deonly.csv"

data = pd.read_csv(filepath)
answerdata = pd.read_csv(answerpath)


Werfen wir einen Blick auf die rohen Daten:

In [ ]:
data

In [ ]:
answerdata

Um hieraus gute Abbildungen mit Matplotlib zu bauen brauchen wir einen Zähler für die Antwortmöglichkeiten:

In [ ]:
def aggregate(col, d):
    counter = [0,0,0,0,0,0,0,0]
    for ans in d[col]:
        match ans:
            case "AO12":
                counter[1] += 1
            case "AO22":
                counter[2] += 1
            case "AO32":
                counter[3] += 1
            case "AO42":
                counter[4] += 1
            case "AO52":
                counter[5] += 1
            case "AO62":
                counter[6] += 1
            case "AO72":
                counter[7] += 1
            case _: 
                counter[0] += 1
    return [i for i in counter if i != 0]
                

Beispiel des output der "aggregate" fuktion

In [ ]:
aggregate("G01Q01", data)

Zwischen-schritt Auflistung der Frage-Codes, die einfach verarbeitet werden können

In [ ]:
q_aggregate_codes = ["G01Q01", "G01Q02", "G01Q03", "G01Q04", "G01Q05", "G01Q06", "G01Q07", "G01Q08", "G01Q09", "G01Q10", "G01Q11", "G01Q12", "G01Q13", "G01Q14", "G01Q15", "G01Q16"]

Wir bauen nun aus diesen einfachen Abfragen erste Darstellungen: 

In [ ]:
for code in q_aggregate_codes:
    count = aggregate(code, data)
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text(code + "\n" + answerdata[code][0])
    # plt.xticks(rotation=90)
    # plt.xticks(wrap=True)
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc)
    labels = ["No Answer"]
    for i in range(len(count)-1):
        labels.insert(len(labels), "Answer {}".format((i + 1)))      
    palette = sns.color_palette("inferno", len(count))
    rects = ax.bar(labels, count, label=answers, color=palette)
    # ax.bar(answers, count, color=palette, label=answers)
    ax.legend(loc='best')
    # annotate rects
    _counter = 0
    for rect in rects:
        val = perc_counts[_counter]
        p_val = round(val*100, 1)
        ax.annotate("{}%".format(p_val),
                    xy = (rect.get_x() + rect.get_width() / 2, rect.get_height()),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        _counter += 1         
    
    plt.savefig(cwdir + "\\out\\png\\bar\\" + code + ".png", bbox_inches="tight")
    plt.show()

    # also do pie-charts
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text(code + "\n" + answerdata[code][0])
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    """perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc) """ 
    palette = sns.color_palette("inferno", len(count))
    ax.pie(count, labels=answers, labeldistance=None, colors = palette,  autopct = '%.2f')

    plt.legend()
    plt.savefig(cwdir + "\\out\\png\\pies\\" + code + ".png", bbox_inches="tight")
    plt.show()
    
    # fig = go.Figure([go.Bar(x=answers, y=count, legend='legend1')])
    # fig.to_html(cwdir + "/out/html/" + code + "/.html")
    # fig.show()


print(code, count, answers, possible_answers)

Nach diesen einfachen geht es nun zunächst an die spezifischere Auswertung. Uns Interessieren hier vor allem die Antworten der Mitglieder lokaler Mensen: dass heisst derer, die regelmäßig im Zeughaus, in der Triplex, in der Zentralmensa.

Beginnen wir mit der Zeughaus-Mensa: wir möchten also einen Datensatz aller, die bei der Frage G01Q03 "zeughaus-Mensa im Marstall", ergo AO12, antworteten:

In [ ]:
marstall_mensa_roh = [[]]
for index, row in data.iterrows():
    if row["G01Q03"] == "AO12":
        marstall_mensa_roh.append(row)

Wir konvertieren die generierte Liste einer Liste in ein Data-Frame und betrachten die Daten. Tatsächlich sind die Antworten bei G01Q03, wie erwartet für AO12 == 100%

In [ ]:
marstall_mensa_data = pd.DataFrame(marstall_mensa_roh, columns=["Antwort ID", "letzte Seite", "Sprache", "Zufallsnummer", "G01Q01", "G01Q02", "G01Q03", "G01Q04", "G01Q05", "G01Q06", "G01Q07", "G01Q08", "G01Q09", "G01Q10", "G01Q11", "G01Q12", "G01Q13", "G01Q14", "G01Q15", "G01Q16", "G01Q017", "G01Q018"])
marstall_mensa_data

Wir können nun das gleiche Spiel von oben auch bei den Daten der Marstallmensa seperat ausführen.

In [ ]:
for code in q_aggregate_codes:
    count = aggregate(code, marstall_mensa_data)
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zeughaus | " + code + "\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]

    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc)
    labels = ["No Answer"]
    for i in range(len(count)-1):
        labels.insert(len(labels), "Answer {}".format((i + 1)))
    palette = sns.color_palette("inferno", len(count))
    rects = ax.bar(labels, count, label=answers, color=palette)
    # ax.bar(answers, count, color=palette, label=answers)
    ax.legend(loc='best')
    # annotate rects
    _counter = 0
    for rect in rects:
        val = perc_counts[_counter]
        p_val = round(val*100, 1)
        ax.annotate("{}%".format(p_val),
                    xy = (rect.get_x() + rect.get_width() / 2, rect.get_height()),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        _counter += 1

    plt.savefig(cwdir + "\\out\\png\\bar\\Zeughaus\\" + code + ".png", bbox_inches="tight")
    plt.show()

    # also do pie-charts
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zeughaus | " + code + "\n" + answerdata[code][0])
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    """perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc) """
    palette = sns.color_palette("inferno", len(count))
    ax.pie(count, labels=answers, labeldistance=None, colors = palette,  autopct = '%.2f')

    plt.legend()
    plt.savefig(cwdir + "\\out\\png\\pies\\Zeughaus\\" + code + ".png", bbox_inches="tight")
    plt.show()

    print(code, count, answers, possible_answers)
    

Wir tun dies nun auch für die anderen zwei Mensen:

In [ ]:
triplex_mensa_roh = [[]]
for index, row in data.iterrows():
    if row["G01Q03"] == "AO22":
        triplex_mensa_roh.append(row)

zentral_mensa_roh = [[]]
for index, row in data.iterrows():
    if row["G01Q03"] == "AO32":
        zentral_mensa_roh.append(row)

triplex_mensa_data = pd.DataFrame(triplex_mensa_roh, columns=["Antwort ID", "letzte Seite", "Sprache", "Zufallsnummer", "G01Q01", "G01Q02", "G01Q03", "G01Q04", "G01Q05", "G01Q06", "G01Q07", "G01Q08", "G01Q09", "G01Q10", "G01Q11", "G01Q12", "G01Q13", "G01Q14", "G01Q15", "G01Q16", "G01Q017", "G01Q018"])

zentral_mensa_data = pd.DataFrame(zentral_mensa_roh, columns=["Antwort ID", "letzte Seite", "Sprache", "Zufallsnummer", "G01Q01", "G01Q02", "G01Q03", "G01Q04", "G01Q05", "G01Q06", "G01Q07", "G01Q08", "G01Q09", "G01Q10", "G01Q11", "G01Q12", "G01Q13", "G01Q14", "G01Q15", "G01Q16", "G01Q017", "G01Q018"])

for code in q_aggregate_codes:
    count = aggregate(code, triplex_mensa_data)
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Triplex | " + code + "\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]
    
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc)
    labels = ["No Answer"]
    for i in range(len(count)-1):
        labels.insert(len(labels), "Answer {}".format((i + 1)))
    palette = sns.color_palette("inferno", len(count))
    rects = ax.bar(labels, count, label=answers, color=palette)
    # ax.bar(answers, count, color=palette, label=answers)
    ax.legend(loc='best')
    # annotate rects
    _counter = 0
    for rect in rects:
        val = perc_counts[_counter]
        p_val = round(val*100, 1)
        ax.annotate("{}%".format(p_val),
                    xy = (rect.get_x() + rect.get_width() / 2, rect.get_height()),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        _counter += 1

    plt.savefig(cwdir + "\\out\\png\\bar\\Triplex\\" + code + ".png", bbox_inches="tight")
    plt.show()

    # also do pie-charts
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Triplex | " + code + "\n" + answerdata[code][0])
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    """perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc) """
    palette = sns.color_palette("inferno", len(count))
    ax.pie(count, labels=answers, labeldistance=None, colors = palette,  autopct = '%.2f')

    plt.legend()
    plt.savefig(cwdir + "\\out\\png\\pies\\Triplex\\" + code + ".png", bbox_inches="tight")
    plt.show()
    
    
    print(code, count, answers, possible_answers)



for code in q_aggregate_codes:
    count = aggregate(code, zentral_mensa_data)
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zentralmensa | " + code + "\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc)
    labels = ["No Answer"]
    for i in range(len(count)-1):
        labels.insert(len(labels), "Answer {}".format((i + 1)))
    palette = sns.color_palette("inferno", len(count))
    rects = ax.bar(labels, count, label=answers, color=palette)
    # ax.bar(answers, count, color=palette, label=answers)
    ax.legend(loc='best')
    # annotate rects
    _counter = 0
    for rect in rects:
        val = perc_counts[_counter]
        p_val = round(val*100, 1)
        ax.annotate("{}%".format(p_val),
                    xy = (rect.get_x() + rect.get_width() / 2, rect.get_height()),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        _counter += 1

    plt.savefig(cwdir + "\\out\\png\\bar\\Zentral\\" + code + ".png", bbox_inches="tight")
    plt.show()

    # also do pie-charts
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zentralmensa | " + code + "\n" + answerdata[code][0])
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    """perc_counts=[]
    for c in count:
        pc = c / sum(count)
        perc_counts.insert(len(perc_counts), pc) """
    palette = sns.color_palette("inferno", len(count))
    ax.pie(count, labels=answers, labeldistance=None, colors = palette,  autopct = '%.2f')

    plt.legend()
    plt.savefig(cwdir + "\\out\\png\\pies\\Zentral\\" + code + ".png", bbox_inches="tight")
    plt.show()
    
    print(code, count, answers, possible_answers)



Eine wichtige Abfrage wäre die Zählung mit Gewichtung nach Häufigkeit des Besuch. Dies Versuche ich nun hier zu implementieren. Wir nehmen ein paar Dinge hierzu an: A) Wer nicht geantwortet hat will nicht, ergo gewicht = 0; B) wer geantwortet hat aber nie in die Mnesa geht hat eine geringe Bereitschaft in die Mensa zu gehen - ergo Gewicht = .2. C) Wer öfter als fünf mal in der Woche in die Mensa geht geht nicht arg viel öfter als 6x - ergo gewicht = 6.1

Kleine Anmerkung zu `return [i for i in counter if i != 0]`: Es gibt keinen Fall, wo eine Antwort-möglichkeit nicht ausgewählt wurde. Deshalb funktioniert dies *in diesem Fall*

In [ ]:

def aggregate_weighted(col, d, w_col):
    counter = [0,0,0,0,0,0,0,0]
    if w_col == "G01Q02":
        w_col = 5
    for ans in d[col]:
        print(d[col][w_col])
        match d[col][w_col]:
            case "AO12":
                inc = 1
            case "AO22":
                inc = 2
            case "AO32":
                inc = 3.5
            case "AO42":
                inc = 5
            case "AO52":
                inc = 6.1
            case _:
                inc = 0
        match ans:
            case "AO12":
                counter[1] += inc
            case "AO22":
                counter[2] += inc
            case "AO32":
                counter[3] += inc
            case "AO42":
                counter[4] += inc
            case "AO52":
                counter[5] += inc
            case "AO62":
                counter[6] += inc
            case "AO72":
                counter[7] += inc
            case _:
                counter[0] += inc
    return [i for i in counter if i != 0]
    


In [ ]:

q_aggregate_code_weighted_st = ["G01Q01", "G01Q02", "G01Q05"]

for code in q_aggregate_codes:
    count = aggregate_weighted(code, data, "G01Q02")
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text(code + " | Weighted\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    palette = sns.color_palette("inferno", len(count))
    ax.bar(range(0,len(count)), count, label=answers, color=palette)
    ax.legend()
    plt.savefig(cwdir + "\\out\\png\\bar\\weighted\\" + code + ".png", bbox_inches="tight")
    print(code, count, answers, possible_answers)

for code in q_aggregate_codes:
    count = aggregate_weighted(code, marstall_mensa_data, "G01Q02")
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zeughaus | " + code + " | Weighted\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    palette = sns.color_palette("inferno", len(count))
    ax.bar(range(0,len(count)), count, label=answers, color=palette)
    ax.legend()
    plt.savefig(cwdir + "\\out\\png\\bar\\weighted\\Zeughaus\\" + code + ".png", bbox_inches="tight")
    plt.show()
    print(code, count, answers, possible_answers)

for code in q_aggregate_codes:
    count = aggregate_weighted(code, triplex_mensa_data, "G01Q02")
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Triplex | " + code + " | Weigthed\n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    palette = sns.color_palette("inferno", len(count))
    ax.bar(range(0,len(count)), count, label=answers, color=palette)
    ax.legend()
    plt.savefig(cwdir + "\\out\\png\\bar\\weighted\\Triplex\\" + code + ".png", bbox_inches="tight")
    plt.show()
    print(code, count, answers, possible_answers)

for code in q_aggregate_codes:
    count = aggregate_weighted(code, zentral_mensa_data, "G01Q02")
    possible_answers = len(count) - 1
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.title.set_text("Zentralmensa | " + code + " | Weighted \n" + answerdata[code][0])
    plt.xticks(rotation=90)
    answers = ["keine Antwort"]
    for c in range(possible_answers):
        # answers.insert(len(answers), "Answer {}".format(c + 1))
        answers.insert(len(answers), answerdata[code][c + 1])
    palette = sns.color_palette("inferno", len(count))
    ax.bar(range(0,len(count)), count, label=answers, color=palette)
    ax.legend()
    plt.savefig(cwdir + "\\out\\png\\bar\\weighted\\Zentral\\" + code + ".png", bbox_inches="tight")
    plt.show()
    print(code, count, answers, possible_answers)
